# Задание 1

Напишите программу с двумя потоками. В первом потоке выведите все чётные числа, во втором -- нечётные. Нижняя и верхняя граница указывается на входе (можно захардкодить, чтобы не вводить).

Пример:

Входные данные:
```
30
50
```

Ответ:
```
List of even numbers:
30 32 34 36 38 40 42 44 46 48 50
List of odd numbers:
31 33 35 37 39 41 43 45 47 49
```

In [1]:
import threading

def get_number(number):
    while True:
        try:
            value = int(input(number))
            return value
        except ValueError:
            print("Введите целое число")

lower_bound = get_number("Введите нижнюю границу диапазона: ")
upper_bound = get_number("Введите верхнюю границу диапазона: ")

def print_even_numbers():
    print("List of even numbers: ")
    for num in range(lower_bound, upper_bound+1):
        if num % 2 == 0:
            print(num, end = " ")
    print()

def print_odd_numbers():
    print("List of odd numbers: ")
    for num in range(lower_bound, upper_bound+1):
        if num % 2 != 0:
            print(num, end = " ")
    print()

even_thread = threading.Thread(target=print_even_numbers)
odd_thread = threading.Thread(target=print_odd_numbers)

even_thread.start()
odd_thread.start()

even_thread.join()
odd_thread.join()

List of even numbers: 
2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48 50 52 54 56 58 60 62 64 66 68 70 72 74 76 78 80 82 84 86 88 90 92 94 96 98 100 
List of odd numbers: 
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47 49 51 53 55 57 59 61 63 65 67 69 71 73 75 77 79 81 83 85 87 89 91 93 95 97 99 


# Задание 2

Реализуйте программу, которая конкурентно посчитает факториал числа.

Входные данные:
```
12
```

Ответ:
```
479001600
```

In [1]:
from multiprocessing import Process, Queue

def get_number(number):
    while True:
        try:
            value = int(input(number))
            if value >= 0:
                return value
            else: 
                print("Введите неотрицательное целое число!")
        except ValueError:
            print("Введите целое число")

def factorial(n, queue):
    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
    queue.put(factorial)

if __name__ == "__main__":
    n = get_number("Введите число для расчета факториала: ")
    queue = Queue()

    factorial_process = Process(target=factorial, args=(n, queue))
    factorial_process.start()
    factorial_process.join()

    # Получаем результат из очереди
    if not queue.empty():
        print(queue.get())

# Задание 3

Напишите 2 функции: первая считает квадрат числа (можно взять готовые реализации), вторая -- сумму чисел.

Каждую из указанных функций запустите в отдельном процессе. В результате у вас должен получится пайплайн, который получает на вход числа от 1 до $n$ (включительно), считает для них квадрат, и для полученных чисел считает сумму.

Входные данные:
```
10
```

Ответ:
```
385
```

In [11]:
import multiprocessing

def calc_squares(numbers, conn):
    conn.send([num ** 2 for num in numbers])
    conn.close()

def calc_sum(conn):
    print(sum(conn.recv()))

def get_number(number):
    while True:
        try:
            value = int(input(number))
            return value
        except ValueError:
            print("Введите целое число")

if __name__ == "__main__":
    multiprocessing.set_start_method("spawn", force=True)
    n = get_number("Введите число:")

    numbers = list(range(1, n + 1))

    parent_conn, child_conn = multiprocessing.Pipe()

    square_process = Process(target=calc_squares, args=(numbers, child_conn))
    sum_process = Process(target=calc_sum, args=(parent_conn,))
    square_process.start()
    sum_process.start()

    square_process.join()
    sum_process.join()

    while parent_conn.poll():
        print(parent_conn.recv(), end= " ")

# Задание 4

Реализуйте параллельный алгоритм сортировки слиянием (merge sort) с использованием multiprocessing.Array либо multiprocessing.Manager.

Входные данные:
```
3 14 15 9 2 6 5 35
```

Ответ:
```
2 3 5 6 9 14 15 35
```

In [ ]:
from multiprocessing import Process, Manager

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

def parallel_merge_sort(array, result):
    if len(array) <= 1:
        result.extend(array) 
        return

def parallel_merge_sort(array, result):
    if len(array) <= 1:
        result.extend(array)
        return
    
    mid = len(array) // 2  # Находим середину массива
    left_result = Manager().list()  # Список для результата левой половины
    right_result = Manager().list()  # Список для результата правой половины

    left_process = Process(target=parallel_merge_sort, args=(array[:mid], left_result))
    right_process = Process(target=parallel_merge_sort, args=(array[mid:], right_result))

    left_process.start()
    right_process.start()

    left_process.join()
    right_process.join()

    result.extend(merge(left_result, right_result))

if __name__ == "__main__":
    array = list(map(int, input("Введите числа через пробел: ").split()))
    
    result = Manager().list()

    parallel_merge_sort(array, result)

    print("Отсортированный массив:", list(result))